In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re
from pathlib import Path

In [2]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y_%m_%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y_%m_%d')

print(today_y_m_d)

2022_09_23


In [3]:
cycle = 1
date_now = pd.Timestamp.today()
# - timedelta(days=1)
if cycle in [1, 2]:
        task_for_call = list(range(1, 34))
else:
    task_for_call = list(range(1, 13)) + list(range(14, 20)) + [22, 23, 24, 25]
ther_exclude_dates_june = [str(val)[:10] for val in pd.date_range('2022-01-01', '2022-06-07')]
ther_exclude_dates_july = [str(val)[:10] for val in pd.date_range('2022-01-01', '2022-07-04')]
ther_exclude_dates_august = [str(val)[:10] for val in pd.date_range('2022-01-01', '2022-08-01')]
date_ymd = date_now.strftime('%Y%m%d')
yesterday_date_ = date_now - dt.timedelta(days=1)

In [4]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = os.path.join(Path.home(), r'Documents\GitHub\final_framework\merged_statuses')
patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_path = os.path.join('C:\\', r'\\', *patients_current_statuses_path)
print(patients_current_statuses_path)

C:\\Users\VAIO\Documents\GitHub\final_framework\merged_statuses


In [5]:
import sys
path_to_pdn = os.path.join(Path.home(), 'Documents\GitHub\pdn___')
# сюда внутри кавычек вставляем путь, где ледит репозиторий пдн
sys.path.insert(0, path_to_pdn)

In [6]:
from pdn.projects.emias_data import get_emias_data
from pdn.reading.read_files import get_group_name_df

In [7]:
import sys
path_to_pdn = os.path.join(Path.home(), 'Documents\GitHub\pdn_') # сюда внутри кавычек вставляем путь, где ледит репозиторий пдн
sys.path.insert(0, path_to_pdn)

In [8]:
from pdn.projects.emias_data import get_emias_id_and_task_num_and_uschedule_visit

In [9]:
from pdn.projects.emias_data import get_status_rec_rcp_procedure_and_doc

In [10]:
# cловарь "номер_dat": "месяц", в который мы добавляем список месяцев, которые сейчас в работе

dat_month = dict()

# dat_month['may'] = '3196'
# dat_month['june'] = '3437'
# dat_month['july'] = '3582'
dat_month['august'] = 3833
# dat_month['august_additional'] = '3940'
dat_month['september'] = 4059

In [11]:
group_names = get_group_name_df()

In [12]:
date_now

Timestamp('2022-09-23 10:31:40.196437')

In [13]:
%%time
for month, dat in dat_month.items():
    
    # пациенты и группы
    emias_data = get_emias_data(date_now, dat)
    for col in [
        'kont_priyom_terapevt_zapis',
        'kont_priyom_terapevt_sdel',
        'kont_priyom_terapevt_sdel_hron_plan'
    ]:
        emias_data.loc[emias_data[col].isin(ther_exclude_dates_august), [col]] = np.nan
    attrs_emias_data = emias_data.merge(
        get_emias_id_and_task_num_and_uschedule_visit(
            emias_data,
            date_now
        ),
        how='left'
    ).merge(
        get_status_rec_rcp_procedure_and_doc(
            emias_data,
            date_now
        ),
        how='left'
    )
    
    
    to_merge = attrs_emias_data[['emiasid', 'group', 'procedure_rec_status']].merge(group_names, on=['group']).rename(
    columns={'procedure_rec_status': 'group_procedure', 'patient_group': 'group_name'})
    
    patients_current_statuses = pd.read_excel(
        os.path.join(patients_current_statuses_path, rf'{month}_patients_current_statuses_with_calls_info_{today_y_m_d}.xlsx'))
    
    to_merge['emiasid'] = to_merge['emiasid'].astype(str).str.replace('\.0', '')
    patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
    
    patients_current_statuses_merged_with_groups = patients_current_statuses.merge(to_merge, on=['emiasid'], how='left')
    
    patients_current_statuses_merged_with_groups['emiasid'] = \
    patients_current_statuses_merged_with_groups['emiasid'].astype(str).str.replace('\.0', '') 
    
        
    patients_current_statuses_merged_with_groups.to_excel(os.path.join(
        patients_current_statuses_path, 
                     rf'{month}_patients_current_statuses_with_groups_{today_y_m_d}.xlsx'), index=False)

C:\Users\VAIO\Documents\GitHub\pdn___\pdn\data_preprocess\preprocess_emias_data.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  melt_data['variable'] = melt_data['variable'].str.replace('_sdel|_zapis', '')
C:\Users\VAIO\Documents\GitHub\pdn___\pdn\data_preprocess\preprocess_emias_data.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  melt_data['variable'] = melt_data['variable'].str.replace('_sdel|_zapis', '')
C:\Users\VAIO\Documents\GitHub\pdn___\pdn\data_preprocess\preprocess_emias_data.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  melt_data['variable'] = melt_data['variable'].str.replace('_sdel|_zapis', '')
C:\Users\VAIO\Documents\GitHub\pdn___\pdn\projects\emias_data.py:198: FutureWarning: The default value of regex will change from True to False in a future version.
  data_attr_by_pat[col] = data_attr_by_pat[col].astyp

CPU times: total: 4min 56s
Wall time: 5min 1s
